<a href="https://colab.research.google.com/github/sayanghorui/samplecode/blob/master/word2vec_practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
train = pd.read_csv("/content/drive/My Drive/ML_datasets/word2vec-nlp-tutorial/labeledTrainData.tsv", 
                    header=0, delimiter="\t",quoting=3)
test = pd.read_csv("/content/drive/My Drive/ML_datasets/word2vec-nlp-tutorial/testData.tsv", 
                    header=0, delimiter="\t",quoting=3)
unlabeled_train = pd.read_csv("/content/drive/My Drive/ML_datasets/word2vec-nlp-tutorial/unlabeledTrainData.tsv", 
                    header=0, delimiter="\t",quoting=3)

In [0]:
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews" 
      % (train["review"].size, test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews


In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
      
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    # 5. Return a list of words
    return(words)

In [0]:
import nltk.data
nltk.download('punkt') 

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords ))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [0]:
print(len(sentences))

795538


In [0]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


In [0]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
# model_name = "300features_40minwords_10context"
model.save("/content/drive/My Drive/ML_datasets/300features_40minwords_10context")

2020-01-10 14:11:43,194 : INFO : 'pattern' package not found; tag filters are not available for English
2020-01-10 14:11:43,204 : INFO : collecting all words and their counts
2020-01-10 14:11:43,208 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-01-10 14:11:43,258 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2020-01-10 14:11:43,314 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2020-01-10 14:11:43,369 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2020-01-10 14:11:43,427 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2020-01-10 14:11:43,490 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2020-01-10 14:11:43,550 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2020-01-10 14:11:43,605 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2020-01-10 14:11:43,661 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2020-01-10 14:11:43,722 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2020-01-10 14:11:43,782 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2020-01-10 14:11:43,840 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2020-01-10 14:11:43,899 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

In [0]:
model.doesnt_match("france england germany berlin".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'berlin'

In [0]:
model.doesnt_match("paris berlin london austria".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'paris'

In [0]:
model.most_similar("happy")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('satisfied', 0.535423219203949),
 ('afraid', 0.45269709825515747),
 ('lucky', 0.44025880098342896),
 ('pleased', 0.43810075521469116),
 ('grateful', 0.4347759187221527),
 ('proud', 0.4189411699771881),
 ('ready', 0.4110080599784851),
 ('thankful', 0.4036791920661926),
 ('happier', 0.40216904878616333),
 ('glad', 0.4007124900817871)]

In [0]:
model.similarity("happy","beautiful")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.10380092

In [0]:
from gensim.models import Word2Vec
model = Word2Vec.load("/content/drive/My Drive/ML_datasets/300features_40minwords_10context")

2020-01-10 14:47:19,553 : INFO : loading Word2Vec object from /content/drive/My Drive/ML_datasets/300features_40minwords_10context
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2020-01-10 14:47:20,016 : INFO : loading wv recursively from /content/drive/My Drive/ML_datasets/300features_40minwords_10context.wv.* with mmap=None
2020-01-10 14:47:20,017 : INFO : setting ignored attribute vectors_norm to None
2020-01-10 14:47:20,018 : INFO : loading vocabulary recursively from /content/drive/My Drive/ML_datasets/300features_40minwords_10context.vocabulary.* with mmap=None
2020-01-10 14:47:20,021 : INFO : loading trainables recursively from /content/drive/My Drive/ML_datasets/300features_40

In [0]:
model.syn0_lockf.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0_lockf` (Attribute will be removed in 4.0.0, use self.trainables.vectors_lockf instead).
  """Entry point for launching an IPython kernel.


(16490,)